## TF-IDF

In [35]:
import os
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [36]:
import pandas as pd
from datasets import load_dataset
from dataset import Dataset
from transformers import T5Tokenizer, BartTokenizer
pd.set_option('display.max_colwidth', None)

#### TEST TFIDF

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
text_tfidf = ['my name is beau', 'my name is fabby', 'i love deep learning']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True) #use_idf bool, default=True (to highlight by comparison) Enable inverse-document-frequency reweighting
x = tfidf_vectorizer.fit_transform(text_tfidf)
print(x.toarray().shape)
# print(tfidf_vectorizer.get_feature_names().shape)
#         tfidfcounts = pd.DataFrame(x.toarray(),index = tfidf_vectorizer.get_feature_names(), columns = ["tfidf"])
tfidfcounts = pd.DataFrame(x.toarray(),index = text_tfidf,  columns = tfidf_vectorizer.get_feature_names())
tfidfcounts

(3, 5)


,beau,deep,fabby,learning,love
my name is beau,1.0,0.00000,0.0,0.00000,0.00000
my name is fabby,0.0,0.00000,1.0,0.00000,0.00000
i love deep learning,0.0,0.57735,0.0,0.57735,0.57735


### Load dataset

In [48]:
# Load dataset
data = 'xsum'

if data == 'cnn_dailymail':
    dataset = load_dataset(data, '3.0.0')
    source_text = "article"
    target_text = "highlights"
elif data == "xsum":
    dataset = load_dataset(data)
    source_text = "document"
    target_text = "summary"
else:
    raise ValueError("Undefined dataset")

Using custom data configuration default
Reusing dataset xsum (/home/pranisaa/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

### Get dataset

In [39]:
model_params = {
    "MODEL": "bart-base",  # model_type: t5-base/t5-large
    "BATCH_SIZE": 1,  # training batch size
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 36,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

train_dataset = dataset["train"]
print("Total training set len: ", len(train_dataset))

# Define portion due to RAM limitation
portion = 6
tfidf_data_size = len(train_dataset)//portion

tokenizer = BartTokenizer.from_pretrained(f'facebook/{model_params["MODEL"]}') # just because our Dataset requires it
training_set = Dataset(
    train_dataset[: tfidf_data_size],
    tokenizer,
    model_params["MODEL"],
    model_params["MAX_SOURCE_TEXT_LENGTH"],
    model_params["MAX_TARGET_TEXT_LENGTH"],
    source_text,
    target_text,
)
print("Final data set len: ", len(training_set))

Total training set len:  204045
Final data set len:  34007


In [40]:
df_tfidf = training_set.compute_tfidf()
df_tfidf_mean = df_tfidf.mean(axis=0).nlargest(500)

/home/pranisaa/thesis/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [42]:
df_tfidf_mean

said           0.037405
mr             0.019586
year           0.016670
people         0.015084
police         0.015080
                 ...   
development    0.002504
growth         0.002499
performance    0.002498
michael        0.002498
issues         0.002492
Length: 500, dtype: float64

In [49]:
to_mask_list = []
for i,v in enumerate(df_tfidf_mean):
#     print(df_tfidf_mean.index[i])
#     print(v)
    to_mask_list.append(df_tfidf_mean.index[i])
#     print(df_tfidf_mean.index[i])

In [50]:
# to_mask_list

In [45]:
# import required modules
import spacy
nlp = spacy.load("en_core_web_sm")


nouns = []
verbs = []
adjs = []
advs = []
cardinals = []
others = []
for word in to_mask_list:
  
    # returns a document of object
    tag = nlp(word)
    if "NN" in tag[0].tag_ :
        nouns.append(word)
    elif "VB" in tag[0].tag_ :
        verbs.append(word)
    elif "JJ" in tag[0].tag_ :
        adjs.append(word)
    elif "RB" in tag[0].tag_ :
        advs.append(word)
    elif "CD" in tag[0].tag_ :
        cardinals.append(word)
    else:
        others.append((word,tag[0].tag_))
#     # checking if it is a noun or not
#     if(tag[0].tag_ == 'NNP'):
#         print(text, " is a noun.")
# else:
#     print(text, " is not a noun.")

In [46]:
print("len of nouns: ", len(nouns))
print("len of verbs: ", len(verbs))
print("len of adjs: ", len(adjs))
print("len of others: ", len(others))

len of nouns:  276
len of verbs:  117
len of adjs:  52
len of others:  6


# Spacy Tag
'''
    ".": "punctuation mark, sentence closer",
    ",": "punctuation mark, comma",
    "-LRB-": "left round bracket",
    "-RRB-": "right round bracket",
    "``": "opening quotation mark",
    '""': "closing quotation mark",
    "''": "closing quotation mark",
    ":": "punctuation mark, colon or ellipsis",
    "$": "symbol, currency",
    "#": "symbol, number sign",
    "AFX": "affix",
    "CC": "conjunction, coordinating",
    "CD": "cardinal number",
    "DT": "determiner",
    "EX": "existential there",
    "FW": "foreign word",
    "HYPH": "punctuation mark, hyphen",
    "IN": "conjunction, subordinating or preposition",
    "JJ": "adjective (English), other noun-modifier (Chinese)",
    "JJR": "adjective, comparative",
    "JJS": "adjective, superlative",
    "LS": "list item marker",
    "MD": "verb, modal auxiliary",
    "NIL": "missing tag",
    "NN": "noun, singular or mass",
    "NNP": "noun, proper singular",
    "NNPS": "noun, proper plural",
    "NNS": "noun, plural",
    "PDT": "predeterminer",
    "POS": "possessive ending",
    "PRP": "pronoun, personal",
    "PRP$": "pronoun, possessive",
    "RB": "adverb",
    "RBR": "adverb, comparative",
    "RBS": "adverb, superlative",
    "RP": "adverb, particle",
    "TO": 'infinitival "to"',
    "UH": "interjection",
    "VB": "verb, base form",
    "VBD": "verb, past tense",
    "VBG": "verb, gerund or present participle",
    "VBN": "verb, past participle",
    "VBP": "verb, non-3rd person singular present",
    "VBZ": "verb, 3rd person singular present",
    "WDT": "wh-determiner",
    "WP": "wh-pronoun, personal",
    "WP$": "wh-pronoun, possessive",
    "WRB": "wh-adverb",
    "SP": "space (English), sentence-final particle (Chinese)",
    "ADD": "email",
    "NFP": "superfluous punctuation",
    "GW": "additional word in multi-word expression",
    "XX": "unknown",
    "BES": 'auxiliary "be"',
    "HVS": 'forms of "have"',
    "_SP": "whitespace",
'''

#### Save

In [47]:
import pickle

with open("to_mask_list_top500", "wb") as fp:   #Pickling
    pickle.dump(to_mask_list, fp)

### Result Evaluation

T5-small xsum
- get samples with low rouge scores
- get samples with high rouge scores
- tf-idf